# Introduction

This tutorial shows how to classify terpenes using classic as well as deep learning classification methods.

The terpenes data is a subset of the [COCONUT](https://coconut.naturalproducts.net) dataset (version March 2021), which is obtained by filtering COCONUT's ``chemicalSuperClass`` column to include only ``Lipids and lipid-like molecules``.

# Setup

In [1]:
import napr
from napr.data import load_terpene
from napr.apps import Terpene

napr.__version__

Data preprocessing finished in 0h:00m:1s.
Dimension reduction finished in 0h:00m:0s.


,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,chemicalSubClass
42937,2.271991,-5.742865,1.996220,-2.540460,-1.358706,-1.207865,-1.176524,-0.490332,-1.047839,0.416635,0.312684,-0.203861,-1.015862,0.402428,Diradylglycerols
344415,-4.762928,0.934697,-1.716314,-1.404995,-1.380104,0.725578,1.545996,-0.492713,-0.230339,0.400925,0.534990,-0.523752,0.368263,0.436824,Monoterpenoids
288712,-0.600844,1.198201,-0.519183,-0.381188,-1.066453,-2.020922,0.539754,-0.931507,-0.138369,-0.149004,0.337500,0.140364,-0.466132,-0.170394,Oxosteroids
317805,-0.525496,1.459377,2.500530,-1.885367,1.642551,0.106756,0.568270,-0.610136,0.210782,-0.285930,0.127464,0.165763,0.182164,-0.176034,Triterpenoids
162280,5.665109,2.466193,-1.858931,-0.291969,-0.045238,0.842174,0.063308,-0.542546,0.625657,0.042450,0.267172,0.398666,-0.922232,-0.233424,Terpene glycosides
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227606,-5.695763,-2.162702,-1.726022,-0.917404,0.978630,-0.088247,-1.585362,1.131933,1.054700,1.460334,0.832248,0.781640,-0.129552,0.814061,Fatty acid esters
56220,-0.929136,2.387381,2.418671,-1.606931,1.381628,0.495785,-0.898296,0.386816,-0.489577,-0.775206,0.049995,-0.566773,-0.011373,0.142681,Triterpenoids
269486,-0.054994,2.802222,1.243644,-1.403711,1.373321,0.350333,-0.637587,0.439211,-0.516168,-0.787450,0.405398,-0.713566,-0.284804,0.051313,Triterpenoids
349333,5.576723,-7.528642,1.678236,-2.127703,-0.279378,0.140734,2.098989,-0.715169,1.778943,0.203275,0.480164,-0.205572,-0.323324,0.110646,Glycerophosphocholines


# Data

In [ ]:
# Download and load the terpene dataset.
# Note: the dataset, 'terpene-21.3.bz2', is saved by default to the current
# directory.
data = load_terpene(download=True)  # load_terpene() if downloaded already

data

# Preprocessing

In [ ]:
terpene = Terpene(data=data)
terpene.preprocess()

terpene.data

# Classification using classic methods

# Classification using deep learning